In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Testing Assumptions

Usually, you will start with empirical data.

You might have educated guesses about the distributions, but you are often not sure.

Let's see how to test this.

## Preprocess
First, we preprocess the penguins data so that we only have the floating point data for continuous distributions

Jupyter notebook will search in our kernel for packages, but it won't know where to find our own src package. But we can add our current working directory with `"."` to `sys.path`, which lists all the locations where python will try to find packages.

If you are curious, just print `sys.path` to have a look.

In [ ]:
import sys
sys.path.insert(0, ".")
from src.settings import modelsettings
from src import preprocess

In [ ]:
modelsettings

In [ ]:
filename = ("../.." / modelsettings.processed_dir / "processed.parq").resolve()
p = preprocess.prepare_floats(filename, modelsettings)

Now, we have melted and normalized data.


## Visualize

In [ ]:

plt.figure(figsize=(10,5))
sns.boxplot(data = p.to_pandas(), x = 'variable', y='norm', hue='species')

Are this normal distributions?

In [ ]:

sns.kdeplot(p.to_pandas(), x="norm", hue="variable")

Maybe, maybe not...

In [ ]:
g = sns.FacetGrid(p.to_pandas(), col="variable", hue="species")
g.map_dataframe(sns.kdeplot, x="norm")

Regardless of the type of distribution, we can still indicate how easy it will be to separate, for example, species.

## Testing distributions

Let's select, from the species "Adelie", the length of the flipper.


In [ ]:
vars = p.select(pl.col("variable").unique())["variable"].to_list()
vars

In [ ]:
x = p.filter((pl.col("variable") == "fliplen") & (pl.col("species") == "Adelie"))["norm"].to_list()

Now we can use the `Fitter` library to fit a distribution.
Let's test these four distrubutions:

In [ ]:

from fitter import Fitter, get_common_distributions
d = ["uniform", "lognorm", "norm", "gamma"] 
d


Now we fit, and make a summary.

In [ ]:
f = Fitter(x, distributions=d)
f.fit()
f.summary()

You can also get the parameters of the best fit:

In [ ]:
f.get_best()

Or from all of them:

In [ ]:
f.fitted_param

Fitter uses the pylab backend hardcoded. But we can hack the fitter object, by studying the [source code](https://fitter.readthedocs.io/en/latest/_modules/fitter/fitter.html#Fitter).

- There is a `self.get_best()` method that provides the parameters of the best fit.
- `self._data` provides us with the raw data for the historgram
- We can use `self.df_errors` to get a dataframe of the best fits. If we sort it, we can obtain the top 5 distributions.
- `plt.plot(f.x, f.fitted_pdf[name])` gives us the fitted pdf. `name` should be the name of the distribution.


In [ ]:
from src import visualization
fig, ax = plt.subplots()
visualization.custom_summary(f, ax)

This gives almost the same result, but we have more control over the details.

As you can see, the distributions are sort of close. But most probable, a normal distribution in this case.

Another, more informal way to check, is with a qq-plot (quantile-quantile plot) to visually test how the samples compare to the theoretical output for a normal distribution.

In [ ]:
from scipy import stats
stats.probplot(x, dist="norm", plot=plt);

You can see there are still some outliers at the end. A perfect fit would line up along the red line.

Wrapping this:

In [ ]:

visualization.test_distribution(x)

That is confirmative for a normal distribution.
Now, let's try the same for one of the bloodlevel values, $\Delta 13$.

In [ ]:
x = p.filter((pl.col("variable") == 'Δ13') & (pl.col("species") == "Adelie"))["norm"].to_list()
f = visualization.test_distribution(x)

As you can see, the normal distribution is not even in the top 5!

You should be carefull, however, to just pick the number 1...

In [ ]:
f.df_errors.sort_values(by="sumsquare_error")

As you can see, the top 3 is pretty close. A gamma distribution has 4 parameters, so that is what makes it really flexible. 

A lognormal could be a good pick here, but I would prefer to dive deeper into the underlying groups. It could be that the distribution returns to two normal distributions, for example if you split the data into male/female.

Also, I would want to talk to an expert about what is going on, to find out if this makes sense.

However, the most important conclusion: this is NOT a normal distribution, and you can't use anything that assumes a normal distribution!